<a href="https://colab.research.google.com/github/Diishasing/Social-Media-Recommendation-system/blob/main/recomsys_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#we have to predict the rating as part of recommendation system

IMPORT PACKAGES


In [ ]:
from IPython.display import Image
import random
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.io import loadmat
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

LOADING DATA

In [ ]:
rating_info = loadmat('/content/rating.mat')['rating']
trust_info = loadmat('/content/trustnetwork.mat')['trustnetwork']

In [ ]:
rating_info

array([[    1,     1,     3,     2],
       [    1,     2,     2,     2],
       [    1,     3,     3,     2],
       ...,
       [22166, 43538,     5,     5],
       [22166, 38711,     3,     4],
       [22166, 41790,     5,     3]], dtype=int32)

In [ ]:
trust_info

array([[15373,  9831],
       [ 4247,  9831],
       [ 4644,  9831],
       ...,
       [13181, 15645],
       [  897,  8000],
       [ 8000,   897]], dtype=uint16)

TRAIN | VALID | TEST DATA

In [ ]:
rating_list = []
trust_list = []

u_items_list = []
u_users_list = []
u_users_items_list = []
i_users_list = []

user_count = 0
item_count = 0
rate_count = 0


In [ ]:
for x in rating_info:
    uid = x[0]
    iid = x[1]
    label = x[2]
    user_count = max(user_count, uid)
    item_count = max(item_count, iid)
    rate_count = max(rate_count, label)
    rating_list.append([uid, iid, label])

In [ ]:
pos_list = []
for x in rating_list:
    pos_list.append((x[0], x[1], x[2]))

In [ ]:
user_count, item_count, rate_count

(22166, 296277, 27)

In [ ]:
pos_list

[(1, 1, 3),
 (1, 2, 2),
 (1, 3, 3),
 (1, 4, 3),
 (1, 5, 3),
 (1, 6, 11),
 (1, 7, 3),
 (1, 8, 3),
 (1, 9, 3),
 (1, 10, 3),
 (2, 11, 3),
 (2, 12, 3),
 (2, 13, 3),
 (2, 14, 3),
 (2, 15, 5),
 (2, 16, 5),
 (2, 1, 3),
 (2, 17, 11),
 (2, 18, 12),
 (2, 19, 3),
 (2, 20, 4),
 (2, 21, 10),
 (2, 22, 10),
 (2, 23, 3),
 (2, 24, 11),
 (2, 25, 3),
 (2, 26, 3),
 (2, 27, 3),
 (2, 28, 3),
 (2, 29, 3),
 (2, 30, 2),
 (2, 31, 10),
 (2, 32, 13),
 (2, 33, 3),
 (2, 34, 10),
 (2, 35, 10),
 (2, 36, 10),
 (2, 37, 10),
 (2, 38, 3),
 (2, 39, 14),
 (2, 40, 10),
 (2, 41, 4),
 (3, 42, 9),
 (3, 43, 13),
 (3, 44, 9),
 (3, 45, 15),
 (3, 46, 15),
 (3, 47, 15),
 (3, 48, 2),
 (3, 49, 11),
 (3, 50, 10),
 (3, 51, 1),
 (3, 52, 11),
 (3, 53, 10),
 (3, 54, 11),
 (3, 55, 11),
 (3, 56, 11),
 (3, 57, 15),
 (4, 58, 5),
 (4, 59, 3),
 (4, 60, 5),
 (4, 61, 5),
 (4, 62, 5),
 (4, 63, 4),
 (4, 64, 5),
 (4, 65, 5),
 (4, 66, 5),
 (4, 67, 5),
 (4, 68, 7),
 (4, 65, 5),
 (4, 69, 10),
 (4, 70, 10),
 (4, 71, 17),
 (5, 72, 4),
 (5, 73, 4),
 (5, 7

In [ ]:
pos_list = list(set(pos_list))
random.shuffle(pos_list)
num_test = int(len(pos_list) * 0.1)
test_set = pos_list[:num_test]
valid_set = pos_list[num_test:2 * num_test]
train_set = pos_list[2 * num_test:]
print(len(train_set), len(valid_set), len(test_set))

730054 91256 91256


In [ ]:
#saving the data in pickle file
with open('./dataset.pkl', 'wb') as f:
    pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(valid_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)


In [ ]:
train_df = pd.DataFrame(train_set, columns = ['uid', 'iid', 'label'])
valid_df = pd.DataFrame(train_set, columns = ['uid', 'iid', 'label'])
test_df = pd.DataFrame(train_set, columns = ['uid', 'iid', 'label'])

click_df = pd.DataFrame(rating_list, columns = ['uid', 'iid', 'label'])
train_df = train_df.sort_values(axis = 0, ascending = True, by = 'uid')


In [ ]:
train_df.head

<bound method NDFrame.head of           uid     iid  label
398971      1       8      3
482767      1       3      3
709951      1       5      3
45853       1       2      2
630661      1      10      3
...       ...     ...    ...
549979  22166   41819      5
26005   22166   41771      5
431931  22166  230287      5
690312  22166   41790      5
465690  22166   80536      3

[730054 rows x 3 columns]>

In [ ]:
max(train_df['label'].values.tolist()), min(train_df['label'].values.tolist())

(27, 1)

PREPARING USER-ITEM LIST

In [ ]:
for user in tqdm(range(user_count + 1)):
    user_df = train_df[train_df['uid'] == user]
    user_items = user_df['iid'].tolist()
    user_ratings = user_df['label'].tolist()
    if len(user_items) == 0:
        u_items_list.append([(0, 0)])
    else:
        u_items_list.append([(iid, rating) for iid, rating in zip(user_items, user_ratings)])    

100%|██████████| 22167/22167 [00:22<00:00, 981.81it/s] 


In [ ]:
u_items_list

[[(0, 0)],
 [(8, 3),
  (3, 3),
  (5, 3),
  (2, 2),
  (10, 3),
  (9, 3),
  (7, 3),
  (1, 3),
  (6, 11),
  (4, 3)],
 [(35, 10),
  (21, 10),
  (31, 10),
  (15, 5),
  (18, 12),
  (29, 3),
  (20, 4),
  (24, 11),
  (19, 3),
  (25, 3),
  (14, 3),
  (37, 10),
  (13, 3),
  (36, 10),
  (12, 3),
  (17, 11),
  (22, 10),
  (34, 10),
  (41, 4),
  (40, 10),
  (38, 3),
  (39, 14),
  (23, 3),
  (1, 3),
  (28, 3),
  (32, 13),
  (16, 5),
  (33, 3),
  (26, 3)],
 [(51, 1),
  (50, 10),
  (54, 11),
  (45, 15),
  (57, 15),
  (46, 15),
  (48, 2),
  (56, 11),
  (55, 11),
  (49, 11),
  (42, 9),
  (44, 9),
  (43, 13),
  (52, 11),
  (47, 15)],
 [(67, 5),
  (68, 7),
  (69, 10),
  (63, 4),
  (70, 10),
  (65, 5),
  (58, 5),
  (71, 17),
  (62, 5),
  (61, 5),
  (66, 5),
  (59, 3),
  (60, 5)],
 [(79, 19),
  (74, 4),
  (77, 17),
  (82, 19),
  (72, 4),
  (73, 4),
  (83, 19),
  (75, 19),
  (80, 19),
  (76, 4)],
 [(94, 15),
  (112, 15),
  (106, 17),
  (113, 10),
  (107, 15),
  (115, 15),
  (99, 10),
  (97, 2),
  (87, 15),
 

PREPARE ITEM-USER LIST


In [ ]:
for item in tqdm(range(item_count + 1)):
    item_df = train_df[train_df['iid'] == item]
    item_users = item_df['uid'].tolist()
    item_ratings = item_df['label'].tolist()
    if len(item_users) == 0:
        i_users_list.append([(0, 0)])
    else:
        i_users_list.append([(uid, rating) for uid, rating in zip(item_users, item_ratings)])
            

100%|██████████| 296278/296278 [05:03<00:00, 974.81it/s] 


In [ ]:
i_users_list

[[(0, 0)],
 [(1, 3),
  (2, 3),
  (41, 3),
  (61, 3),
  (429, 3),
  (533, 3),
  (561, 3),
  (649, 3),
  (668, 3),
  (798, 3),
  (829, 3),
  (894, 3),
  (997, 3),
  (1018, 3),
  (1086, 3),
  (1107, 3),
  (1174, 3),
  (1248, 3),
  (1338, 3),
  (1426, 3),
  (1444, 3),
  (1580, 3),
  (1836, 3),
  (1875, 3),
  (1913, 3),
  (1943, 3),
  (1992, 3),
  (2014, 3),
  (2057, 3),
  (2125, 3),
  (2456, 3),
  (2667, 3),
  (2691, 3),
  (2884, 3),
  (3151, 3),
  (3175, 3),
  (3252, 3),
  (3286, 3),
  (3306, 3),
  (3307, 3),
  (3371, 3),
  (3373, 3),
  (3421, 3),
  (3532, 3),
  (3636, 3),
  (3637, 3),
  (3667, 3),
  (3699, 3),
  (3883, 3),
  (3978, 3),
  (4111, 3),
  (4232, 3),
  (4311, 3),
  (4409, 3),
  (4763, 3),
  (4822, 3),
  (4953, 3),
  (5104, 3),
  (5290, 3),
  (5472, 3),
  (6173, 3),
  (6204, 3),
  (6303, 3),
  (6388, 3),
  (6431, 3),
  (6476, 3),
  (6561, 3),
  (6620, 3),
  (6621, 3),
  (6666, 3),
  (6691, 3),
  (6715, 3),
  (6722, 3),
  (6728, 3),
  (6732, 3),
  (6798, 3),
  (6832, 3),
  (6894

In [ ]:
#processing the trust information
for x in trust_info:
    uid = x[0]
    fid = x[1]
    if uid > user_count or fid > user_count:
        continue
    trust_list.append([uid, fid])    


In [ ]:
trust_df = pd.DataFrame(trust_list, columns = ['uid', 'fid'])
trust_df = trust_df.sort_values(axis = 0, ascending = True, by = 'uid')
trust_df.head

<bound method NDFrame.head of           uid    fid
283312      2   7659
84889       2    829
221672      2  21911
247685      2   7336
306637      2   6339
...       ...    ...
285484  22164   5864
283826  22164  11427
285362  22164  10897
135466  22165   2198
263608  22165  14953

[355754 rows x 2 columns]>

In [ ]:
for user in tqdm(range(user_count + 1)):
    user_df = trust_df[trust_df['uid'] == user]
    u_users = user_df['fid'].unique().tolist()
    if len(u_users) == 0:
        u_users_list.append([0])
        u_users_items_list.append([[(0, 0)]])
    else:
        u_users_list.append(u_users)
        uu_items = []
        for uid in u_users:
            uu_items.append(u_items_list[uid]) 
        u_users_items_list.append(uu_items)       

100%|██████████| 22167/22167 [00:17<00:00, 1298.46it/s]


In [ ]:
with open('./list_epinions.pkl', 'wb') as f:
    pickle.dump(u_items_list, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(u_users_list, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(u_users_items_list, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(i_users_list, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((user_count, item_count, rate_count), f, pickle.HIGHEST_PROTOCOL)


PREPARE GRAPH DATASET CLASS

In [ ]:
class GraphData(Dataset):
    def __init__(self, data, u_items_list, u_users_list, u_users_items_list, i_users_list):
        self.data = data
        self.u_items_list = u_items_list
        self.u_users_list = u_users_list
        self.u_users_items_list = u_users_items_list
        self.i_users_list = i_users_list

    def __getitem__(self, index):
        uid = self.data[index][0]
        iid = self.data[index][1]
        label = self.data[index][2]
        u_items = self.u_items_list[uid] 
        u_users = self.u_users_list[uid]
        u_users_items = self.u_users_items_list[uid]
        i_users = self.i_users_list[iid]

        return (uid, iid, label), u_items, u_users, u_users_items, i_users  
    def __len__(self):
        return len(self.data)     

In [ ]:
#collate function for padding and truncation

truncate_len = 50
def collate_fn(batch_data):
    uids, iids, labels = [], [], []
    u_items, u_users, u_users_items, i_users = [], [], [], []
    u_items_len, u_users_len, i_users_len = [], [], []

    for data, u_items_u, u_users_u, u_users_items_u, i_users_i in batch_data:
        (uid, iid, label) = data
        uids.append(uid)
        iids.append(iid)
        labels.append(label)

        #user-items
        if len(u_items_u) <= truncate_len:
            u_items.append(u_items_u)
        else:
            u_items.append(random.sample(u_items_u, truncate_len))
        u_items_len.append(min(len(u_items_u), truncate_len))

        #users-users and users-users-items
        if len(u_users_u) <= truncate_len:
            u_users.append(u_users_u)
            u_u_items = []
            for uui in u_users_items_u:
                if len(uui) < truncate_len:
                    u_u_items.append(uui)
                else:
                    u_u_items.append(random.sample(uui, truncate_len)) 

            u_users_items.append(u_u_items)

        else:
           sample_index = random.sample(list(range(len(u_users_u))), truncate_len)
           u_users.append([u_users_u[si] for si in sample_index]) 

           u_users_items_u_tr = [u_users_items_u[si] for si in sample_index]
           u_u_items = []

           for uui in u_users_items_u_tr:
               if len(uui) < truncate_len:
                   u_u_items.append(uui) 
               else:
                   u_u_items.append(random.sample(uui, truncate_len))
           u_users_items.append(u_u_items)

        u_users_len.append(min(len(u_users_u), truncate_len))

        #items-users
        if len(i_users_i) <= truncate_len:
            i_users.append(i_users_i)
        else:
            i_users.append(random.sample(i_users_i, truncate_len)) 
        i_users_len.append(min(len(i_users_i), truncate_len))

    batch_size = len(batch_data)           

    #padding
    u_items_maxlen = max(u_items_len)
    u_users_maxlen = max(u_users_len)
    i_users_maxlen = max(i_users_len)

    u_item_pad = torch.zeros([batch_size, u_items_maxlen, 2], dtype = torch.long)
    for i, ui in enumerate(u_items):
        u_item_pad[i, :len(ui), :] = torch.LongTensor(ui)

    u_user_pad = torch.zeros([batch_size, u_users_maxlen, 2], dtype = torch.long)
    for i, uu in enumerate(u_users):
        u_user_pad[i, :len(ui), :] = torch.LongTensor(ui)    

    u_user_item_pad = torch.zeros([batch_size, u_users_maxlen, u_items_maxlen, 2], dtype = torch.long)
    for i, uu_items in enumerate(u_users_items):
        for j, ui in enumerate(uu_items):
            u_user_item_pad[i, j, :len(ui), :] = torch.LongTensor(ui)

    i_user_pad = torch.zeros([batch_size, i_users_maxlen, 2], dtype = torch.long)
    for i, iu in enumerate(i_users):
        i_user_pad[i, :len(iu), :] = torch.LongTensor(iu)

    uids = torch.LongTensor(uids)
    iids = torch.LongTensor(iids)
    labels = torch.LongTensor(labels)

    return uids, iids, labels, u_item_pad, u_user_pad, u_user_item_pad, i_user_pad                




ARCHITECTURE

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, input_dim//2, bias = True),
            nn.ReLU(),
            nn.Linear(input_dim//2, output_dim, bias=True)
        )
    def forward(self, x):
        return self.mlp(x)

class Aggregator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Aggregator, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, output_dim, bias = True),
            nn.ReLU()
        )      

    def forward(self, x):
        return self.mlp(x)
                 

In [ ]:
class UserModel(nn.Module):
    def __init__(self, emb_dim, user_emb, item_emb, rating_emb):
        super(UserModel, self).__init__()
        self.emb_dim = emb_dim
        self.user_emb = user_emb
        self.item_emb = item_emb
        self.rating_emb = rating_emb

        self.g_v  = MLP(2*self.emb_dim, self.emb_dim)

        self.user_item_attn = MLP(2*self.emb_dim, 1)
        self.aggr_items = Aggregator(self.emb_dim, self.emb_dim)

        self.user_user_attn = MLP(2*self.emb_dim, 1)
        self.aggr_neighbors = Aggregator(self.emb_dim, self.emb_dim)

        self.mlp = nn.Sequential(
            nn.Linear(2*self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU()
        )

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.eps = 1e-10

    def forward(self, uids, u_item_pad, u_user_pad, u_user_item_pad):

        q_a = self.item_emb(u_item_pad[:,:,0])
        u_item_er = self.rating_emb(u_item_pad[:,:,1])
        x_ia = self.g_v(torch.cat([q_a, u_item_er], dim=2).view(-1, 2*self.emb_dim)).view(q_a.size())
        mask_u = torch.where(u_item_pad[:,:,0]>0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        p_i = mask_u.unsqueeze(2).expand_as(x_ia) * self.user_emb(uids).unsqueeze(1).expand_as(x_ia)
        alpha = self.user_item_attn(torch.cat([x_ia, p_i], dim=2).view(-1, 2*self.emb_dim)).view(mask_u.size())
        alpha = torch.exp(alpha)*mask_u
        alpha = alpha / (torch.sum(alpha, 1).unsqueeze(1).expand_as(alpha) + self.eps)
        h_iI = self.aggr_items(torch.sum(alpha.unsqueeze(2).expand_as(x_ia) * x_ia, 1))


        q_a_s = self.item_emb(u_user_item_pad[:,:,:,0])
        u_user_item_er = self.rating_emb(u_user_item_pad[:,:,:,1])
        x_ia_s = self.g_v(torch.cat([q_a_s, u_user_item_er], dim=2).view(-1, 2*self.emb_dim)).view(q_a_s.size())
        mask_s = torch.where(u_user_item_pad[:,:,:,0]>0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        p_i_s = mask_s.unsqueeze(3).expand_as(x_ia_s) * self.user_emb(u_user_pad).unsqueeze(2).expand_as(x_ia_s)
        alpha_s = self.user_item_attn(torch.cat([x_ia_s, p_i_s], dim=3).view(-1, 2*self.emb_dim)).view(mask_s.size())
        alpha_s = torch.exp(alpha_s)*mask_s
        alpha_s = alpha_s / (torch.sum(alpha_s, 2).unsqueeze(2).expand_as(alpha_s) + self.eps)
        h_oI_temp = torch.sum(alpha_s.unsqueeze(3).expand_as(x_ia_s) * x_ia_s, 2)
        h_oI = self.aggr_items(h_oI_temp.view(-1, self.emb_dim)).view(h_oI_temp.size())
        
        beta = self.user_user_attn(torch.cat([h_oI, self.user_emb(u_user_pad)], dim = 2).view(-1, 2 * self.emb_dim)).view(u_user_pad.size())
        mask_su = torch.where(u_user_pad > 0, torch.tensor([1.], device=self.device), torch.tensor([0.], device=self.device))
        beta = torch.exp(beta) * mask_su
        beta = beta / (torch.sum(beta, 1).unsqueeze(1).expand_as(beta) + self.eps)
        h_iS = self.aggr_neighbors(torch.sum(beta.unsqueeze(2).expand_as(h_oI) * h_oI, 1))

        h_i = self.mlp(torch.cat([h_iI, h_iS], dim = 1))

        return h_i

In [ ]:
class ItemModel(nn.Module):
    def __init__(self, emb_dim, user_emb, item_emb, rating_emb):
        super(ItemModel, self).__init__()
        self.emb_dim = emb_dim
        self.user_emb = user_emb
        self.item_emb = item_emb
        self.rating_emb = rating_emb 

        self.g_u = MLP(2*self.emb_dim, self.emb_dim)

        self.item_users_attn = MLP(2*self.emb_dim, 1)
        self.aggr_users = Aggregator(self.emb_dim, self.emb_dim)

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.eps = 1e-10

    def forward(self, iids, i_user_pad):

        p_t = self.user_emb(i_user_pad[:, :, 0])
        i_user_er = self.rating_emb(i_user_pad[:, :, 1])
        mask_i = torch.where(i_user_pad[:, :, 0] > 0, torch.tensor([1.], device = self.device), torch.tensor([0.], device = self.device))
        f_jt = self.g_u(torch.cat([p_t, i_user_er], dim = 2).view(-1, 2 * self.emb_dim)).view(p_t.size())
        q_j = mask_i.unsqueeze(2).expand_as(f_jt) * self.item_emb(iids).unsqueeze(1).expand_as(f_jt)
        mu_jt = self.item_users_attn(torch.cat([f_jt, q_j], dim = 2).view(-1, 2 * self.emb_dim)).view(mask_i.size())
        mu_jt = torch.exp(mu_jt) * mask_i 
        mu_jt = mu_jt / (torch.sum(mu_jt, 1).unsqueeze(1).expand_as(mu_jt) + self.eps)

        z_j = self.aggr_users(torch.sum(mu_jt.unsqueeze(2).expand_as(f_jt) * f_jt ))

        return z_j  

In [ ]:
class RecomModule(nn.Module):
    def __init__(self, n_users, n_items, n_ratings, emb_dim = 64):
        super(RecomModule, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.n_ratings = n_ratings
        self.emb_dim = emb_dim

        self.user_emb = nn.Embedding(self.n_users, self.emb_dim, padding_idx = 0)
        self.item_emb = nn.Embedding(self.n_items, self.emb_dim, padding_idx = 0)
        self.rating_emb = nn.Embedding(self.n_ratings, self.emb_dim, padding_idx = 0)

        self.user_model = UserModel(self.emb_dim, self.user_emb, self.item_emb, self.rating_emb)
        self.item_model = ItemModel(self.emb_dim, self.user_emb, self.item_emb, self.rating_emb)

        self.mlp = nn.Sequential(
            nn.Linear(2*self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, self.emb_dim, bias = True),
            nn.ReLU(),
            nn.Linear(self.emb_dim, 1)
        )

    def forward(self, uids, iids, u_item_pad, u_user_pad, u_user_item_pad, i_user_pad):

        h_j = self.user_model(uids, u_item_pad, u_user_pad, u_user_item_pad)
        z_j = self.user_model(iids, i_user_pad)

        r_ij = self.mlp(torch.cat([h_j, z_j], dim = 1))

        return r_ij  


HYPERPARAMETERS

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device = ' + str(device))
batch_size = 128
embed_dim = 64
learning_rate = 0.001
epochs = 2

device = cuda


In [ ]:
with open('./dataset.pkl', 'rb') as f:
    train_set = pickle.load(f)
    valid_set = pickle.load(f)
    test_set = pickle.load(f)

with open('./list_epinions.pkl', 'rb') as f:
    u_items_list = pickle.load(f)
    u_users_list = pickle.load(f)
    u_users_items_list = pickle.load(f)
    i_users_list = pickle.load(f)
    (user_count, item_count, rate_count) = pickle.load(f)

In [ ]:
train_data = GraphData(train_set, u_items_list, u_users_list, u_users_items_list, i_users_list)
valid_data = GraphData(valid_set, u_items_list, u_users_list, u_users_items_list, i_users_list)
test_data = GraphData(test_set, u_items_list, u_users_list, u_users_items_list, i_users_list)

DATALOADER


In [ ]:
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True, collate_fn = collate_fn)
valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle = True, collate_fn = collate_fn)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = True, collate_fn = collate_fn)

In [ ]:
print(len(train_loader))
print(len(valid_loader))
print(len(test_loader))

5704
713
713


In [ ]:
for i in train_loader:
    for j in i:
        print(j)
    break    

tensor([20809, 13086, 12445,  9551,  7954, 19057, 11958, 11428, 10390, 21419,
         6876, 17723, 12165, 14056,  5676,  7673,  6665, 14969, 11289, 15699,
         2087, 12285,  2328, 16956,  8407, 15827,  8304, 18704, 13514, 10348,
         7510,  4366, 12503,  5283, 16532, 10531, 20560,  3263,  2760,  1176,
         5658,  2132, 21548, 15445, 17835,  1509,  3849, 15699,  2235, 18752,
        14399,  1155,  9284, 18583, 20987, 21590, 18858,   374,  9003, 12021,
        13658,  3755,  7760, 19681,  7770, 20287, 14465,  8002,  1638, 11408,
        12135,  3574, 16025,  9305,  7596,  5479,   666, 14462, 20813,  1717,
        16024, 14805,  8875, 18277,  9381, 14193, 11159,   349, 13181, 22014,
        13180, 11421,  8596,  2266,  7636, 10567, 14806, 17167,  6387, 20425,
         5711,  6144,  7150, 15489,  8329, 13183,  5957, 16160,  4504, 19116,
         8068, 20511, 21938,  3374,  3631, 12419,  3761, 17913, 13410, 10437,
        16801, 15073, 14826, 19482, 10198,  6668,  6877, 12782])

In [ ]:
user_count, item_count, rate_count

(22166, 296277, 27)

In [ ]:
model = RecomModule(user_count + 1, item_count + 1, rate_count + 1, embed_dim).to(device)

In [ ]:
print(model)

RecomModule(
  (user_emb): Embedding(22167, 64, padding_idx=0)
  (item_emb): Embedding(296278, 64, padding_idx=0)
  (rating_emb): Embedding(28, 64, padding_idx=0)
  (user_model): UserModel(
    (user_emb): Embedding(22167, 64, padding_idx=0)
    (item_emb): Embedding(296278, 64, padding_idx=0)
    (rating_emb): Embedding(28, 64, padding_idx=0)
    (g_v): MLP(
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
      )
    )
    (user_item_attn): MLP(
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=1, bias=True)
      )
    )
    (aggr_items): Aggregator(
      (mlp): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
      )
    )
    (user_user_attn): MLP(
      (mlp): Sequential(
        (0): Linear(in_features

In [ ]:
optimizer = torch.optim.RMSprop(model.parameters(), learning_rate)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 4, gamma = 0.1)

In [ ]:
for epoch in range(epochs):
    #training_step
    model.train()
    s_loss = 0
    for i, (uids, iids, labels, u_items, u_users, u_users_items, i_users) in tqdm(enumerate(train_loader), total = len(train_loader)):
        uids = uids.to(device)
        iids = iids.to(device)
        labels = labels.to(device)
        u_items = u_items.to(device)
        u_users = u_users.to(device)
        u_users_items = u_users_items.to(device)
        i_users = i_users.to(device)

        optimizer.zero_grad()
        outputs = model(uids, iids, u_items, u_users, u_users_items, i_users)
        loss = criterion(outputs, labels.unsqueeze(1))

        loss.backward()
        optimizer.step()

        loss_val = loss.item()
        s_loss += loss_val

        iter_num = epoch * len(train_loader) + i + 1

    #validate step
    model.eval()
    errors = []
    with torch.no_grad():
        for uids, iids, labels, u_items, u_users, u_users_items, i_users in tqdm(valid_loader):
            uids = uids.to(device)
            iids = iids.to(device)    
            labels = labels.to(device)    
            u_items = u_items.to(device)    
            u_users = u_users.to(device)
            u_users_items = u_users_items.to(device)
            i_users = i_users.to(device)
            preds = model(uids, iids, u_items, u_users, u_users_items, i_users)
            error = torch.abs(preds.squeeze(1) - labels)
            errors.extend(error.data.cpu().numpy().tolist())

    mae = np.mean(errors)     
    rmse = np.sqrt(np.mean(np.power(errors, 2))) 

    scheduler.step()

    ckpt_dict = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }      
                
    torch.save(ckpt_dict, './epoch_latest_ckpt.pth')

    if epoch == 0:
        best_mae = mae
    elif mae < best_mae:
        best_mae = mae
        torch.save(ckpt_dict, './epoch_best_ckpt_{}.pth'.format(embed_dim))

    print('epoch : {}, MAE: {:.4f}, RMSE : {:.4f}, Best MAE : {:.4f}'.format(epoch+1, mae, rmse, best_mae))                    



  0%|          | 0/5704 [00:03<?, ?it/s]


RuntimeError: ignored

TEST THE MODEL

In [ ]:
embed_dim = 64
ckpt = torch.load('./latest_ckpt.pth'.format(embed_dim))
model = RecomModule(user_count + 1, item_count + 1, rate_count + 1, embed_dim).to(device)
model.load_state_dict(ckpt['state_dict'])

In [ ]:
#validate step
model.eval()
test_errors = []
with torch.no_grad():
    for uids, iids, labels, u_items, u_users, u_users_items, i_users in tqdm(valid_loader):
        uids = uids.to(device)
        iids = iids.to(device)    
        labels = labels.to(device)    
        u_items = u_items.to(device)    
        u_users = u_users.to(device)
        u_users_items = u_users_items.to(device)
        i_users = i_users.to(device)
        preds = model(uids, iids, u_items, u_users, u_users_items, i_users)
        error = torch.abs(preds.squeeze(1) - labels)
        test_errors.extend(error.data.cpu().numpy().tolist())
test_mae = np.mean(test_errors)     
test_rmse = np.sqrt(np.mean(np.power(test_errors, 2))) 
print(' MAE: {:.4f}, RMSE : {:.4f}'.format(test_mae, test_rmse))                    
